In [2]:
import glob 
import zipfile
import numpy as np
import cv2
import os
import time

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models

import pandas as pd
from sklearn.model_selection import train_test_split


data_root_path = 'drive/My Drive/colab_data/dogs-vs-cats/numpy_folder/'

train_data = []     
def read_data_from_npy_zip():
    for f in tqdm(os.listdir(data_root_path)):
      path = os.path.join(data_root_path, f)
      train_data.extend(np.load(path, allow_pickle=True))

read_data_from_npy_zip()

np.random.shuffle(train_data)
print(len(train_data), train_data[0])

100%|██████████| 26/26 [01:09<00:00,  2.68s/it]

25000 [array([[[ 38,  41,  45],
        [ 38,  41,  45],
        [ 39,  42,  46],
        ...,
        [122, 139, 142],
        [121, 138, 141],
        [121, 138, 141]],

       [[ 38,  41,  45],
        [ 38,  41,  45],
        [ 39,  42,  46],
        ...,
        [128, 145, 148],
        [125, 142, 145],
        [123, 140, 143]],

       [[ 38,  41,  45],
        [ 38,  41,  45],
        [ 39,  42,  46],
        ...,
        [135, 154, 157],
        [131, 149, 152],
        [127, 145, 148]],

       ...,

       [[228, 241, 235],
        [228, 240, 234],
        [217, 229, 223],
        ...,
        [162,  93, 173],
        [161,  92, 172],
        [160,  91, 171]],

       [[221, 233, 227],
        [236, 248, 242],
        [236, 248, 242],
        ...,
        [162,  93, 173],
        [161,  92, 172],
        [159,  90, 170]],

       [[231, 243, 237],
        [236, 248, 242],
        [242, 254, 248],
        ...,
        [160,  91, 171],
        [161,  92, 172],
        [155,  86

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

IMAGE_SIZE = (224, 224)

net = models.resnet18(pretrained=True).to(device)
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
net.fc = nn.Linear(net.fc.in_features, 2)


def run(X, y, is_training=True):
    
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    loss_function = nn.MSELoss()
    
    if is_training:
        net.zero_grad()
        optimizer.zero_grad()
        
    outputs = net(X)
    matches  = [torch.argmax(i)==torch.argmax(j) for i, j in zip(outputs, y)]
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)

    if is_training:
        loss.backward()
        optimizer.step()

    return acc, loss

def batch_and_run(X, y, is_training=True):
    BATCH_SIZE = 64
    for i in tqdm(range(0, len(X), BATCH_SIZE)):
        batch_X = X[i:i+BATCH_SIZE]
        batch_y = y[i:i+BATCH_SIZE]

        print(len(batch_X))
        print(batch_y)
            
        m_X = torch.Tensor([i for i in batch_X])
        m_X = m_X/255.0
        m_y = torch.Tensor([i for i in batch_y])

        m_X = m_X.to(device)
        m_y = m_y.to(device)

        return run(m_X, m_y, is_training)


def train(net):
    EPOCHS = 20  
    for epoch in range(EPOCHS):
        acc, loss = batch_and_run(train_data[0], train_data[1], True)
        #val_acc, val_loss = batch_and_run(X_test, y_test, False)
        print(f"Epoch: {epoch}, {round(time.time(),3)},{round(float(acc),2)},{round(float(loss), 4)}\n")
        
train(net)


predictions=[]
id_line=[]
def test(net):
    for f in os.listdir(test_path):
        id_line.append(f.split('.')[0])
        path = os.path.join(test_path, f)
        img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img, (50, 50))
        test_X = torch.Tensor(img).view(-1, IMAGE_SIZE)
        test_X = test_X.to(device)
        net_out = net(test_X)
        
        predictions.append(torch.argmax(net_out))

test(net)

predicted_val = [x.tolist() for x in predictions]
print(predicted_val)

Running on the GPU


  0%|          | 0/1 [00:00<?, ?it/s]

2
[array([[[ 71, 100, 105],
        [ 69, 100, 105],
        [ 64, 101, 105],
        ...,
        [ 22,  41,  44],
        [ 22,  41,  44],
        [ 25,  42,  45]],

       [[ 73, 102, 107],
        [ 71, 102, 107],
        [ 66, 103, 107],
        ...,
        [ 22,  41,  44],
        [ 23,  42,  45],
        [ 26,  43,  46]],

       [[ 72, 101, 106],
        [ 71, 101, 106],
        [ 65, 102, 106],
        ...,
        [ 22,  41,  45],
        [ 23,  42,  45],
        [ 26,  43,  46]],

       ...,

       [[105, 107, 107],
        [107, 109, 109],
        [110, 112, 112],
        ...,
        [177, 175, 175],
        [177, 175, 175],
        [177, 175, 175]],

       [[107, 109, 109],
        [108, 110, 110],
        [111, 113, 113],
        ...,
        [181, 179, 179],
        [181, 179, 178],
        [181, 179, 178]],

       [[106, 108, 108],
        [107, 109, 109],
        [110, 112, 112],
        ...,
        [184, 182, 181],
        [184, 182, 181],
        [184, 182, 18

ValueError: ignored